In [3]:
from google.colab import drive
drive.mount('/content/drive')
import json
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader


KeyboardInterrupt: ignored

In [ ]:
import json
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [ ]:
from IPython.core.display import JSON
file_path = '/content/drive/MyDrive/Chatbot/intents.json'
#file_path = 'intents.json'


with open(file_path, 'r') as file:
    content = json.load(file)
    print(content)


{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'Hey', 'How are you', 'Is anyone there?', 'Hello', 'Good day'], 'responses': ['Hey :-)', 'Hello, thanks for visiting', 'Hi there, what can I do for you', 'Hi there, how can I help you']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later thanks for visiting', 'Have a nice day', 'By! Come back again soon']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Thanks a lot!'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}, {'tag': 'Items', 'patterns': ['Which items do you have?', 'What Kinds of items are there?', 'What do you sell?'], 'responses': ['we seel coffee and tea', 'We have coffee and tea']}, {'tag': 'payments', 'patterns': ['Do you take credit cards?', 'Do you accept Mastercard?', 'Can I pay with paypal?', 'Are you cash only?'], 'responses': ['We accept VISA , Mastercard and Paypal', 'We accept most major credit cards, and paypal']}, {'tag': 'de

In [ ]:
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()




In [ ]:
def tokenize(sentence):
  return nltk.word_tokenize(sentence)



a = "a quick brown fox jumps over the lazy dog"
print(a)
a = tokenize(a)
print(a)

a quick brown fox jumps over the lazy dog
['a', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']


In [ ]:
def stem(word):
  return stemmer.stem(word.lower())

words = ["Organize", "organizes", "ORGanizing"]
stemmed_words = [stem(w) for w in words]
print(stemmed_words)

['organ', 'organ', 'organ']


In [ ]:
def bag_of_words(tokenized_sentence, all_words):
  """
  sentence = ["hello", "how", "are", "you"]
  words =    ["hi", "hello", "I", "you", "bye","thank", "cool"]
  bag =      [ 0,    1,   0,   1,   0,   0,   0]
  """

  tokenized_sentence = [stem(w) for w in tokenized_sentence]

  bag = np.zeros(len(all_words), dtype=np.float32)
  for idx , w in enumerate(all_words):
    if w in tokenized_sentence:
      bag[idx]=1.0

  return bag





In [ ]:


all_words = []
tags = []
xy = []


In [ ]:
for intent in content['intents']:
  tag = intent['tag']
  tags.append(tag)
  for pattern in intent['patterns']:
    w = tokenize(pattern)
    all_words.extend(w)
    xy.append((w, tag))


ignore_words = ['?', '!', '.']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))
print(all_words)
print(tags)


["'s", 'a', 'accept', 'anyon', 'are', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'deliveri', 'do', 'doe', 'funni', 'get', 'good', 'goodby', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'is', 'item', 'joke', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'me', 'my', 'of', 'onli', 'pay', 'paypal', 'see', 'sell', 'ship', 'someth', 'take', 'tell', 'thank', 'that', 'there', 'what', 'when', 'which', 'with', 'you']
['Items', 'delivery', 'funny', 'goodbye', 'greeting', 'payments', 'thanks']


In [ ]:
X_train = []
y_train = []


for (pattern_sentence, tag) in xy:
  bag = bag_of_words(pattern_sentence, all_words)
  X_train.append(bag)

  label = tags.index(tag)
  y_train.append(label)

X_train= np.array(X_train)
y_train= np.array(y_train)


print(X_train.shape)
print(y_train.shape)





(26, 54)
(26,)


In [ ]:
class ChatDataset(Dataset):
    def __init__(self, X, y):
        self.n_samples = len(X)
        self.x_data = X
        self.y_data = y

    def __getitem__(self, index):
        x = self.x_data[index]
        y = self.y_data[index]
        return x, y

    def __len__(self):
        return self.n_samples

In [ ]:
dataset = ChatDataset(X_train, y_train)

In [ ]:
# hyperparameters
batch_size = 8
learning_rate = 0.001
num_epochs = 1000
hidden_size = 8
output_size = len(tags)
input_size = len(X_train[0])
print(input_size, len(all_words))
print(output_size, tags)


54 54
7 ['Items', 'delivery', 'funny', 'goodbye', 'greeting', 'payments', 'thanks']


In [ ]:
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=2)



###Train Model

In [ ]:
class NeuralNet(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(NeuralNet, self).__init__()
    self.l1 = nn.Linear(input_size, hidden_size)
    self.l2 = nn.Linear(hidden_size, hidden_size)
    self.l3 = nn.Linear(hidden_size, num_classes)
    self.relu = nn.ReLU()

  def forward(self, x):
    out = self.l1(x)
    out = self.relu(out)
    out = self.l2(out)
    out = self.relu(out)
    out = self.l3(out)
    # no activation and no softmax
    return out





In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
model = NeuralNet(input_size, hidden_size, output_size).to(device)


###Loss and optimizer

In [ ]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
  for (words, labels) in train_loader:
    words = words.to(device)
    labels = labels.to(device)


    #forward
    outputs = model(words)
    Loss = loss(outputs, labels)


    #backward and optimizer steps
    optimizer.zero_grad()
    Loss.backward()
    optimizer.step()

  if (epoch +1) % 100==0:
    print(f'epoch{epoch+1}/{num_epochs}, Loss = {Loss.item():.4f}')


print(f'final loss,loss = {Loss.item():.4f}')






epoch100/1000, Loss = 1.1398
epoch200/1000, Loss = 0.1174
epoch300/1000, Loss = 0.1087
epoch400/1000, Loss = 0.0210
epoch500/1000, Loss = 0.0127
epoch600/1000, Loss = 0.0056
epoch700/1000, Loss = 0.0007
epoch800/1000, Loss = 0.0029
epoch900/1000, Loss = 0.0008
epoch1000/1000, Loss = 0.0014
final loss,loss = 0.0014


In [ ]:
data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "output_size": output_size,
    "hidden_size": hidden_size,
    "all_words": all_words,
    "tags": tags
}
FILE = "/content/drive/MyDrive/Parameter.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

training complete. file saved to /content/drive/MyDrive/Parameter.pth


In [ ]:
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

data = torch.load(FILE)
print(data)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data["all_words"]
tags = data["tags"]
model_state = data["model_state"]
model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()


{'model_state': OrderedDict([('l1.weight', tensor([[-0.1238, -0.4205,  0.3105,  0.3708,  0.8344, -0.5801,  0.3289,  0.5599,
          0.1140,  0.4738,  0.7218, -0.3037,  0.3585, -0.4746, -0.1890, -0.3160,
          0.6085, -0.6248,  0.4762,  0.9070, -0.2926,  0.7618,  0.9104,  0.6113,
          0.0721,  0.5736,  0.6261, -0.4836,  0.4416, -0.3919, -0.4843, -0.4351,
         -0.1261,  0.3081, -0.2049, -0.1186,  0.3496, -0.0582,  0.3649,  0.3002,
         -0.3631,  0.6111, -0.2527, -0.0613, -0.0293, -0.1765, -0.6308, -0.1546,
          0.6526,  0.7267, -0.3222,  0.4561,  0.3964,  0.4548],
        [ 0.0402, -0.5218, -0.1987,  0.4150,  0.2692,  0.8821, -0.0062,  0.0037,
         -0.4128, -0.0248,  0.4037, -0.2825,  0.1063, -0.3576, -0.1598, -0.2301,
          0.3162,  0.8129,  0.4564,  0.4607, -0.0991,  0.5842,  0.5228,  0.2306,
         -0.2189,  0.2398,  0.5967, -0.3012,  0.2507, -0.5095,  0.4510, -0.4488,
          0.0853, -0.2885, -0.3329, -0.1869,  0.2202, -0.2615, -0.0374, -0.0581,
  

NeuralNet(
  (l1): Linear(in_features=54, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=7, bias=True)
  (relu): ReLU()
)

In [ ]:
bot_name = "Shara"
print("Let's chat! type 'quit' to exit")
while True:
  sentence = input('You: ')
  if sentence == "quit":
    break
  sentence = tokenize(sentence)
  X = bag_of_words(sentence, all_words)
  X = X.reshape(1, X.shape[0])
  X = torch.from_numpy(X).to(device)  # Assuming 'device' is the desired device (CPU or GPU)

  output = model(X)
  _, predicted = torch.max(output, dim=1)
  tag = tags[predicted.item()]

  probs = torch.softmax(output, dim=1)
  prob = probs[0] [predicted.item()]

  if prob.item() > 0.75:
    for intent in content["intents"]:
      if tag == intent["tag"]:
         print(f"{bot_name}: {random.choice(intent['responses'])}")

  else:
    print(f"{bot_name}: I do not understand...")





Let's chat! type 'quit' to exit
You: hi
Shara: Hi there, what can I do for you
You: what's up
Shara: I do not understand...
You: quit


In [ ]:
import torch

# Assuming 'model' is your PyTorch model
#model_path = '/content/drive/MyDrive/data.pth'  # Replace with your desired file path and name

# Save the PyTorch model to Google Drive
torch.save(model.state_dict(), "FILE")
